### DataFrame과 Dataset
- 결과를 생성하기 위해 어떤 데이터에 어떤 연산을 적용해야하는지 정의하는 지연 연산의 **실행 계획**
- 불변성을 가짐
- 액션을 호출하면 스파크는 실제로 transformation을 실행하고 결과를 반환함
- 스키마: DataFrame의 컬럼명과 데이터 타입을 정의함. 스파크는 자체 데이터 타입을 갖고 있음

- Dataset은 scala와 java에서만 지원되며, Dataset의 경우 컴파일 타임에 스키마에 명시된 데이터 타입의 일치여부를 검증함
- DataFrame은 런타임이 되어서야 데이터 타입의 일치 여부를 검증함
- DataFrame은 Row 타입으로 구성된 Dataset임. 이 형식은 "연산에 최적화된 인메모리 포맷"의 내부적 표현 방식임

### 컬럼과 로우
- 컬럼: 테이블의 컬럼과 동일. 단순 데이터 타입, 배열과 같은 복합 데이터 타입, null을 표현함
- 로우: 데이터 레코드. 데이터소스에서 얻거나 직접 생성 가능

In [9]:
// 스파크의 덧셈 연산
val df = spark.range(500).toDF("number") //0~499까지의 값이 할당된 500 row의 df 생성
df.select(df.col("number")+10).take(10)  //각 로우에 10을 더함

df: org.apache.spark.sql.DataFrame = [number: bigint]
res8: Array[org.apache.spark.sql.Row] = Array([10], [11], [12], [13], [14], [15], [16], [17], [18], [19])


In [12]:
// scala에서 스파크 데이터 타입 사용
import org.apache.spark.sql.types._

val b = ByteType

import org.apache.spark.sql.types._
b: org.apache.spark.sql.types.ByteType.type = ByteType


### 구조적 API의 실행 과정
- DF/DS/SQL을 이용해 코드 작성 &rarr; 논리적 실행 계획 생성 &rarr; 최적화 및 물리적 실행 계획으로 변환 &rarr; 연산 실행
- 논리적 실행 계획: catalyst optimizer가 실행 계획을 최적화
- 물리적 실행 계획: 논리적 실행 계획을 클러스터 환경에서 실행하는 방법 정의. 최종적으로 RDD transformation으로 변환되어 실행

### 구조적 API의 기본 기능
- 스키마: DataFrame의 컬럼명과 데이터 타입을 정의. 스키마를 데이터에서 읽어오거나 스스로 만들 수도 있음
- 사용자는 표현식을 사용하여 컬럼의 선택, 조작, 제거를 수행함. 컬럼은 사실상 표현식으로 레코드의 값을 단순하게 나타내는 논리적 구조임
- 표현식: DataFrame 레코드의 여러 값에 대한 transformation 집합
- row: 스파크는 레코드를 Row 객체로 표현함. Row 객체는 스키마 정보를 갖고 있지 않음

In [21]:
// DataFrame 생성
val df = spark.read.format("json")
  .load("c:/SparkDG/data/flight-data/json/2015-summary.json")

df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [22]:
//스키마 출력
df.printSchema

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [16]:
//column 생성: col 함수 사용
import org.apache.spark.sql.functions.{col, column, expr}
col("someColumnName")
expr("someColumnName2")  // expr 함수에 인수로 컬럼 표현식을 넣으면 col 함수와 동일하게 사용 가능

import org.apache.spark.sql.functions.{col, column, expr}
res14: org.apache.spark.sql.Column = someColumnName2


In [26]:
//row 생성 및 데이터 접근. row 생성시 df와 같은 순서로 값을 명시해야 함
import org.apache.spark.sql.Row
val myRow = Row("Hello", null, 1, false)

myRow.getString(0)
myRow.getInt(2)

import org.apache.spark.sql.Row
myRow: org.apache.spark.sql.Row = [Hello,null,1,false]
res21: Int = 1


2020-08-08 18:03:57 WARN  Executor:87 - Issue communicating with driver in heartbeater
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [10 seconds]. This timeout is controlled by spark.executor.heartbeatInterval
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:36)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:92)
	at org.apache.spark.executor.Executor.org$apache$spark$executor$Executor$$reportHeartBeat(Executor.scala:785)
	at org.apache.spark.executor.Executor$$anon$2$$anonfun$run$1.apply$mcV$sp(Executor.scala:814)
	at org.apache.spark.exe

### select 메서드
